# Easy Money - Modelo de propensión a compra

Mensaje de Carol: Una vez que hayamos hecho el análisis de nuestra BBDD nos gustaría poder vender más y mejor nuestros productos, sé que los data scientist tienen una “bola de cristal” y saben qué clientes son más propensos a comprar nuestros productos así que te pido por favor hagas un modelo de propensión a compra, se que tenemos muchos productos, así que te dejo a tu criterio científico qué productos elegir, o pueden ser varios, ya me sorprenderás

Vamos a desarrollar un modelo capaz de **predecir si un cliente es propenso a contratar un producto de Easy Money, teniendo en cuenta sus características**.

## Importación de librerías

In [84]:
# Importamos las librerias necesarias.
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder

## Importación de datos

De las tablas que tenemos disponibles en BBDD, las que serán de utilidad para este modelo son customer_sociodemographics, customer_commercial_activity y sales. 
<p>Guardamos los datos de las tablas en dataframes distintos que limpiaremos, y posteriormente uniremos en un único dataframe 


In [86]:
# df_cs = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/customer_sociodemographics.csv', index_col=0)
# df_cca = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/customer_commercial_activity.csv', index_col=0)
# df_sales = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/Recursos  Proyecto final-20240502/datasets_TFM + diccionario/sales.csv', index_col=0)

In [87]:
# df_cs = pd.read_csv(r'C:\Users\Usuario\Desktop\DataScience\Material didáctico\TFM\datasets_TFM + diccionario\customer_sociodemographics.csv', index_col=0)
# df_cca = pd.read_csv(r'C:\Users\Usuario\Desktop\DataScience\Material didáctico\TFM\datasets_TFM + diccionario\customer_commercial_activity.csv', index_col=0)
# df_sales = pd.read_csv(r'C:\Users\Usuario\Desktop\DataScience\Material didáctico\TFM\datasets_TFM + diccionario\sales.csv', index_col=0)

In [94]:
# Obtener el directorio de trabajo actual
current_dir = os.getcwd()

# Ir a la carpeta anterior del directorio actual porque ahí se encuentra la carpeta 'data' donde están todos los ficheros .csv
parent_dir = os.path.dirname(current_dir)

DATA_PATH = os.path.join(parent_dir, 'data/') # Ruta donde están los ficheros .csv

FILE_NAME1 = 'customer_sociodemographics.csv'
FILE1 = os.path.join(DATA_PATH, FILE_NAME1)

FILE_NAME2 = 'customer_commercial_activity.csv'
FILE2 = os.path.join(DATA_PATH, FILE_NAME2)

FILE_NAME3 = 'sales.csv'
FILE3 = os.path.join(DATA_PATH, FILE_NAME3)

print('current path:', current_dir)
print('parent path:', parent_dir)
print('data path:', DATA_PATH)
print('file 1:', FILE1)
print('file 2:', FILE2)
print('file 3:', FILE3)
print('data path:', DATA_PATH)

current path: /Users/orlando/Documents/GitHub/TFM-EasyMoney/Dataframe_modelo
parent path: /Users/orlando/Documents/GitHub/TFM-EasyMoney
data path: /Users/orlando/Documents/GitHub/TFM-EasyMoney/data/
file 1: /Users/orlando/Documents/GitHub/TFM-EasyMoney/data/customer_sociodemographics.csv
file 2: /Users/orlando/Documents/GitHub/TFM-EasyMoney/data/customer_commercial_activity.csv
file 3: /Users/orlando/Documents/GitHub/TFM-EasyMoney/data/sales.csv
data path: /Users/orlando/Documents/GitHub/TFM-EasyMoney/data/


In [89]:
df_cs = pd.read_csv(FILE1, index_col=0)
df_cca = pd.read_csv(FILE2, index_col=0)
df_sales = pd.read_csv(FILE3, index_col=0)

## 1: Data Understanding

### 1.1: Análisis Univariante de datos de las distintos datasets que vamos a usar

#### 1.1.1 Analizamos la tabla customer_sociodemographics

In [90]:
# Datos generales 

df_cs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 8 columns):
 #   Column        Dtype  
---  ------        -----  
 0   pk_cid        int64  
 1   pk_partition  object 
 2   country_id    object 
 3   region_code   float64
 4   gender        object 
 5   age           int64  
 6   deceased      object 
 7   salary        float64
dtypes: float64(2), int64(2), object(4)
memory usage: 409.4+ MB


In [91]:
# Visualización de los datos
df_cs.head()

,pk_cid,pk_partition,country_id,region_code,gender,age,deceased,salary
0,1375586,2018-01,ES,29.0,H,35,N,87218.10
1,1050611,2018-01,ES,13.0,V,23,N,35548.74
2,1050612,2018-01,ES,13.0,V,23,N,122179.11
3,1050613,2018-01,ES,50.0,H,22,N,119775.54
4,1050614,2018-01,ES,50.0,V,23,N,NaN


In [86]:
# Recuento de nulos 

df_cs.isna().sum()

pk_cid                0
pk_partition          0
country_id            0
region_code        2264
gender               25
age                   0
deceased              0
salary          1541104
dtype: int64

De entrada, analizamos la columna "deceased" para eliminar los clientes fallecidos y quedarnos con aquellos que actualmente son reales

In [87]:
df_cs['deceased'].value_counts()

deceased
N    5961849
S       1075
Name: count, dtype: int64

In [88]:
df_cs = df_cs[(df_cs['deceased']  == 'N')]
df_cs['deceased'].value_counts()

deceased
N    5961849
Name: count, dtype: int64

Ahora que nos hemos quedado solamente con los clientes no fallecidos, esta columna no es de mucha utilidad ya que siempre tiene el mismo valor. La podemos eliminar.

In [89]:
del(df_cs['deceased'])

Analizamos también la columna country_id para ver si vale la pena quedarnos con todos los países de la BBDD 

In [90]:
df_cs['country_id'].value_counts()

country_id
ES    5959602
GB        441
FR        225
DE        199
US        195
CH        189
BR         87
BE         81
VE         79
IE         68
MX         58
AT         51
AR         51
PL         49
IT         45
MA         34
CL         30
CN         28
CA         22
GA         17
DO         17
CO         17
RU         17
NO         17
GT         17
SN         17
MR         17
ET         17
CM         17
SA         17
CI         17
QA         17
LU         17
SE         16
DJ         11
PT         11
JM         11
RO          9
HU          8
DZ          7
PE          4
Name: count, dtype: int64

Vemos que los valores de los países distintos a ES son muy pequeños y no aportarán demasiado valor a nuestro modelo. Nos quedamos solamente con los registros que son country_id = ES

In [91]:
df_cs = df_cs[(df_cs['country_id']  == 'ES')]
df_cs['country_id'].value_counts()

country_id
ES    5959602
Name: count, dtype: int64

Ahora que nos hemos quedado solamente con los clientes de ES, esta columna no es de mucha utilidad ya que siempre tiene el mismo valor. La podemos eliminar.

In [92]:
del(df_cs['country_id'])

Según el diccionario de las tablas, region_code es el código de provincia. Este dato sí puede ser relevante, pero vemos que tiene nulos. Como estamos ante un modelo de clasificación binaria, podemos rellenar los nulos con un valor como -1, para que se sean agrupados en un bloque propio.

In [93]:
df_cs['region_code'].fillna(-1, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7344\1372854579.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cs['region_code'].fillna(-1, inplace=True)


La columna gender también tenía nulos, pero son muy pocos en relación al total del dataset. Analizamos cuántos nulos quedan después de los cambios ya realizados y elimnamos estas filas por ser muy pocas

In [94]:
df_cs['gender'].isna().sum()

np.int64(25)

In [95]:
df_cs = df_cs.dropna(subset =['gender'])

Analizamos la columna salary 

In [96]:
df_cs['salary'].value_counts()

salary
451931.22    709
288997.44    546
135522.15    466
128318.52    458
104563.80    436
            ... 
195859.59      1
210493.68      1
128959.86      1
90295.38       1
157753.62      1
Name: count, Length: 257626, dtype: int64

In [97]:
df_cs['salary'].describe()

count    4.420911e+06
mean     1.155810e+05
std      2.000189e+05
min      1.202730e+03
25%      6.141354e+04
50%      8.843742e+04
75%      1.313081e+05
max      2.889440e+07
Name: salary, dtype: float64

Vemos que todos los valores son positivos, que la diferencia entre el salario mayor y el salario menor es muy grande, y que la media por tanto es muy elevada también. Igualmente, según el diccionario de la BBDD, se trata del salario de la unidad familiar y consideramos que este valor es importante a la hora de determinar la contratación. Al haber muchos nulos, y al ser el modelo de clasificación binaria, vamos a asignar a los nulos el valor -1

In [98]:
df_cs['salary'].fillna(-1, inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7344\3059662271.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cs['salary'].fillna(-1, inplace=True)


Transformamos la columna region_code de float a int

In [99]:
df_cs["region_code"] = df_cs["region_code"].astype(int)

Transformamos la columna pk_partition a fecha

In [100]:
df_cs["pk_partition"] = pd.to_datetime(df_cs["pk_partition"])


Finalmente, la tabla de customer_sociodemographics quedará así:

In [101]:
df_cs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5959577 entries, 0 to 13647308
Data columns (total 6 columns):
 #   Column        Dtype         
---  ------        -----         
 0   pk_cid        int64         
 1   pk_partition  datetime64[ns]
 2   region_code   int64         
 3   gender        object        
 4   age           int64         
 5   salary        float64       
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 318.3+ MB


#### 1.1.2 Analizamos la tabla customer_commercial_activity

In [102]:
# Datos generales 

df_cca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 6 columns):
 #   Column           Dtype  
---  ------           -----  
 0   pk_cid           int64  
 1   pk_partition     object 
 2   entry_date       object 
 3   entry_channel    object 
 4   active_customer  float64
 5   segment          object 
dtypes: float64(1), int64(1), object(4)
memory usage: 318.5+ MB


In [103]:
# Visualización de los datos
df_cca.head()

,pk_cid,pk_partition,entry_date,entry_channel,active_customer,segment
0,1375586,2018-01,2018-01,KHL,1.0,02 - PARTICULARES
1,1050611,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
2,1050612,2018-01,2015-08,KHE,0.0,03 - UNIVERSITARIO
3,1050613,2018-01,2015-08,KHD,0.0,03 - UNIVERSITARIO
4,1050614,2018-01,2015-08,KHE,1.0,03 - UNIVERSITARIO


In [104]:
# Recuento de nulos 

df_cca.isna().sum()

pk_cid                  0
pk_partition            0
entry_date              0
entry_channel      133033
active_customer         0
segment            133944
dtype: int64

In [105]:
pd.set_option('display.max_rows', 100)

In [106]:
df_cca['entry_channel'].value_counts(normalize = True)*100

entry_channel
KHE    53.413469
KFC    15.276786
KHQ    10.125061
KAT     7.137080
KHK     3.948564
KHM     3.029062
KHN     1.859966
KFA     1.355428
KHD     1.301894
RED     1.039488
KHL     0.715897
KHF     0.354329
KHO     0.141461
KAZ     0.096657
KHC     0.089899
KBG     0.028508
KEH     0.012487
KHP     0.011853
007     0.010515
013     0.006810
KAG     0.005746
KAS     0.005232
KBZ     0.003396
KAA     0.003311
KAR     0.002693
KAY     0.002556
KAF     0.002110
KAB     0.001630
KFD     0.001561
KCC     0.001527
KAW     0.001115
KAQ     0.000892
KDH     0.000875
KAE     0.000823
KCH     0.000823
KCB     0.000806
KAD     0.000772
KAM     0.000738
KAH     0.000686
KAK     0.000583
KAJ     0.000566
KDT     0.000395
KFF     0.000377
KBO     0.000360
KEY     0.000343
KBE     0.000292
KCL     0.000292
KFK     0.000292
KGC     0.000292
KGN     0.000292
KGX     0.000292
004     0.000292
KFS     0.000292
KBH     0.000292
KCI     0.000292
KDR     0.000274
KFL     0.000223
KBW     0.000206


Vemos muchos valores de Entry Channel que son muy pocos en comparación con los valores mayoritarios. Elegimos los canales de entrada que tengan una proporción mayor al 1% y agrupamos el resto en Others

In [107]:
value_counts = (df_cca['entry_channel'].value_counts(normalize = True, dropna=True)*100)
df_cca['entry_channel'] = df_cca['entry_channel'].apply(lambda x: x if pd.isna(x) or value_counts.get(x, 0) >= 1 else 'Others')

In [108]:
df_cca['entry_channel'].value_counts(normalize = True)*100

entry_channel
KHE       53.413469
KFC       15.276786
KHQ       10.125061
KAT        7.137080
KHK        3.948564
KHM        3.029062
KHN        1.859966
Others     1.513202
KFA        1.355428
KHD        1.301894
RED        1.039488
Name: proportion, dtype: float64

Analizamos la columna segment para ver los valores únicos, sabiendo que también tiene nulos

In [109]:
df_cca['segment'].value_counts()

segment
03 - UNIVERSITARIO    3900166
02 - PARTICULARES     1830875
01 - TOP                97939
Name: count, dtype: int64

Rellenamos NaN de channel y segment con un valor identificativo como 'Not defined'

In [110]:
df_cca["entry_channel"].fillna('Not defined', inplace=True)
df_cca["segment"].fillna('Not defined', inplace=True)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7344\2969524961.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cca["entry_channel"].fillna('Not defined', inplace=True)
C:\Users\Usuario\AppData\Local\Temp\ipykernel_7344\2969524961.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a c

In [111]:
df_cca.isna().sum()

pk_cid             0
pk_partition       0
entry_date         0
entry_channel      0
active_customer    0
segment            0
dtype: int64

In [112]:
df_cca['active_customer'].value_counts()

active_customer
0.0    3557573
1.0    2405351
Name: count, dtype: int64

Transformarmos active_customer de float a int

In [113]:
df_cca["active_customer"] = df_cca["active_customer"].astype(int)

Transformamos las columna pk_partition y entry_date a fecha

In [114]:
df_cca["pk_partition"] = pd.to_datetime(df_cca["pk_partition"])
df_cca["entry_date"] = pd.to_datetime(df_cca["entry_date"])

Eliminamos la columna de entry_date ya que la fecha que nos interesa realmente es la fecha de venta dela tabla sales

In [115]:
del(df_cca['entry_date'])

Finalmente, la tabla de customer_commercial_activity quedará así:

In [116]:
df_cca.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5962924 entries, 0 to 13647308
Data columns (total 5 columns):
 #   Column           Dtype         
---  ------           -----         
 0   pk_cid           int64         
 1   pk_partition     datetime64[ns]
 2   entry_channel    object        
 3   active_customer  int64         
 4   segment          object        
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 273.0+ MB


#### 1.1.3 Analizamos la tabla sales

In [117]:
# Datos generales 

df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   pk_sale     240773 non-null  int64  
 1   cid         240773 non-null  int64  
 2   month_sale  240773 non-null  object 
 3   product_ID  240773 non-null  int64  
 4   net_margin  240773 non-null  float64
dtypes: float64(1), int64(3), object(1)
memory usage: 11.0+ MB


In [118]:
# Visualización de los datos
df_sales.head()

,pk_sale,cid,month_sale,product_ID,net_margin
0,6666,33620,2018-05-01,2335,952.9
1,6667,35063,2018-06-01,2335,1625.2
2,6668,37299,2018-02-01,2335,1279.7
3,6669,39997,2018-02-01,2335,1511.9
4,6670,44012,2018-02-01,2335,1680.3


In [119]:
# Recuento de nulos 

df_sales.isna().sum()

pk_sale       0
cid           0
month_sale    0
product_ID    0
net_margin    0
dtype: int64

La columna pk_sale es el identificador de la venta. Esta columna no va a ser necesaria para el modelo, por lo que la eliminamos.

In [120]:
del(df_sales['pk_sale'])

La clolumna net_margin nos puede ser útil para decidir qué producto elegir para la predicción de nuestro modelo, por lo que de momento la dejamos.

Transformamos la columna month_sale a fecha

In [121]:
df_sales["month_sale"] = pd.to_datetime(df_sales["month_sale"])

Finalmente, la tabla de sales quedará así:

In [123]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 240773 entries, 0 to 240772
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   cid         240773 non-null  int64         
 1   month_sale  240773 non-null  datetime64[ns]
 2   product_ID  240773 non-null  int64         
 3   net_margin  240773 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 9.2 MB


## 2: Construcción del dataset para el modelo

Vamos a hacer un inner merge de todos los datasets, ya que nos interesa agrupar toda la información analizada en un único dataframe, que contenga toda la información por el id de cliente.

In [124]:
df_merged = pd.merge(
    left = df_cca, 
    right = df_cs,
    on = ('pk_cid', 'pk_partition'),
    how = 'inner')

In [129]:
df_merged.head()

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary
0,1375586,2018-01-01,Others,1,02 - PARTICULARES,29,H,35,87218.10
1,1050611,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,35548.74
2,1050612,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,122179.11
3,1050613,2018-01-01,KHD,0,03 - UNIVERSITARIO,50,H,22,119775.54
4,1050614,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,V,23,-1.00


In [130]:
#df_merged = pd.merge(
    #left = df_merged, 
    #right = df_sales,
    #left_on = 'pk_cid',
    #right_on  = 'cid',
    #how = 'inner')

In [131]:
pd.set_option('display.max_rows', 1000)

In [96]:
### Revisar para que nos vaya, no pilla la buena carpeta donde esta data, se mete en dataframe_modelo

print(CWD)
print(DATA_PATH)

df_con_reg = pd.read_csv(os.path.join(DATA_PATH, 'reg_pension_plan.csv'), sep = ';', low_memory=False, index_col=0)

/Users/orlando/Documents/GitHub/TFM-EasyMoney/Dataframe_modelo
/Users/orlando/Documents/GitHub/TFM-EasyMoney/data/


FileNotFoundError: [Errno 2] No such file or directory: '/Users/orlando/Documents/GitHub/TFM-EasyMoney/data/reg_pension_plan.csv'

In [ ]:
df_con_reg = pd.read_csv('/Users/alinaoganesyan/Desktop/Master Data Science/Entregables/Proyecto final/git-proyecto/data/reg_pension_plan.csv', sep = ';')

In [222]:
df_con_reg = pd.read_csv(r'C:\Users\Usuario\Desktop\DataScience\TFM_\TFM-EasyMoney\data/reg_pension_plan.csv', sep = ';')

In [223]:
df_con_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   pk_cid            1048575 non-null  int64 
 1   pk_partition      1048575 non-null  object
 2   reg_pension_plan  1048575 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 24.0+ MB


In [224]:
df_con_reg.head()

,pk_cid,pk_partition,reg_pension_plan
0,15891,01/07/2018,0
1,15891,01/08/2018,0
2,16063,01/11/2018,0
3,16063,01/12/2018,0
4,16063,01/01/2019,0


In [225]:
df_con_reg['pk_partition'] = pd.to_datetime(df_con_reg['pk_partition'], format = '%d/%m/%Y' )

In [226]:
df_con_reg.head()

,pk_cid,pk_partition,reg_pension_plan
0,15891,2018-07-01,0
1,15891,2018-08-01,0
2,16063,2018-11-01,0
3,16063,2018-12-01,0
4,16063,2019-01-01,0


In [227]:
df_con_reg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 3 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   pk_cid            1048575 non-null  int64         
 1   pk_partition      1048575 non-null  datetime64[ns]
 2   reg_pension_plan  1048575 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 24.0 MB


In [228]:
df_con_reg_merged = pd.merge(
    left = df_merged, 
    right = df_con_reg,
    on = ('pk_cid', 'pk_partition'),
    how = 'inner')

In [ ]:
df_con_reg_merged.head(50)

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,1050611,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,35548.74,0
1,1050612,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,122179.11,0
2,1050613,2018-01-01,KHD,0,03 - UNIVERSITARIO,50,H,22,119775.54,0
3,1050614,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,V,23,-1.00,0
4,1050615,2018-01-01,KHE,0,03 - UNIVERSITARIO,45,H,23,22220.04,0
5,1050616,2018-01-01,KHE,0,03 - UNIVERSITARIO,24,H,23,295590.36,0
6,1050617,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,H,23,113316.66,0
7,1050619,2018-01-01,KHE,0,03 - UNIVERSITARIO,20,H,24,-1.00,0
8,1050620,2018-01-01,KHE,0,03 - UNIVERSITARIO,10,H,23,113194.98,0
9,1050621,2018-01-01,KHE,0,03 - UNIVERSITARIO,50,V,23,72575.88,0


## Creamos datasets con compras y no compras de pension plan

In [230]:
pension_plan_purch = df_con_reg_merged.loc[(df_con_reg_merged['reg_pension_plan'] == 1)]

In [231]:
pension_plan_purch

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
71,1050659,2018-01-01,KFC,1,02 - PARTICULARES,7,V,39,-1.00,1
120,1050565,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,33,54141.03,1
140,1050537,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,H,23,214309.14,1
186,1050869,2018-01-01,KHE,1,03 - UNIVERSITARIO,28,V,31,45758.82,1
222,1050920,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,25,-1.00,1
...,...,...,...,...,...,...,...,...,...,...
1046941,1017759,2019-05-01,KFC,1,03 - UNIVERSITARIO,28,H,24,-1.00,1
1046944,1017752,2019-05-01,KFA,1,02 - PARTICULARES,28,V,29,78061.95,1
1047276,1019074,2019-05-01,KAT,1,02 - PARTICULARES,8,H,40,237887.46,1
1047442,1018802,2019-05-01,KAT,1,03 - UNIVERSITARIO,28,H,27,219826.11,1


In [232]:
pension_plan_no_purch = df_con_reg_merged.loc[(df_con_reg_merged['reg_pension_plan'] == 0)]

In [233]:
pension_plan_no_purch['pk_partition'].unique()

<DatetimeArray>
['2018-01-01 00:00:00', '2018-02-01 00:00:00', '2018-03-01 00:00:00',
 '2018-04-01 00:00:00', '2018-05-01 00:00:00', '2018-06-01 00:00:00',
 '2018-07-01 00:00:00', '2018-08-01 00:00:00', '2018-09-01 00:00:00',
 '2018-10-01 00:00:00', '2018-11-01 00:00:00', '2018-12-01 00:00:00',
 '2019-01-01 00:00:00', '2019-02-01 00:00:00', '2019-03-01 00:00:00',
 '2019-04-01 00:00:00', '2019-05-01 00:00:00']
Length: 17, dtype: datetime64[ns]

In [234]:
pension_plan_no_purch 

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,1050611,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,35548.74,0
1,1050612,2018-01-01,KHE,0,03 - UNIVERSITARIO,13,V,23,122179.11,0
2,1050613,2018-01-01,KHD,0,03 - UNIVERSITARIO,50,H,22,119775.54,0
3,1050614,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,V,23,-1.00,0
4,1050615,2018-01-01,KHE,0,03 - UNIVERSITARIO,45,H,23,22220.04,0
...,...,...,...,...,...,...,...,...,...,...
1047514,1018682,2019-05-01,KAT,0,02 - PARTICULARES,28,V,88,35310.57,0
1047515,1018679,2019-05-01,KFA,1,02 - PARTICULARES,28,V,34,61180.11,0
1047516,1018676,2019-05-01,KFA,0,02 - PARTICULARES,28,H,74,63867.48,0
1047517,1018675,2019-05-01,KFC,0,02 - PARTICULARES,29,H,15,-1.00,0


Queremos utilizar las fechas del dataset pension_plan_purch anteriores para montar el dataset final con los datos de clientes.
Ya hemos identificado los 1, solo queremos las fechas. 
Cuando tengamos las fechas, podemos ir a los no-purch y sacar los datos anteriores a esas fechas (datetime).
De aquí tendremos clientes que han comprado pero con su comportamiento o descripción el mes anterior a la compra.




In [235]:
# Creamos una nueva columna con los datos de la pk_partition del mes anterior a la compra desde el data frame de purchase
pension_plan_purch['pk_part_prev_month'] = pension_plan_purch['pk_partition'] - pd.DateOffset(months=1)

C:\Users\Usuario\AppData\Local\Temp\ipykernel_7344\459634169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pension_plan_purch['pk_part_prev_month'] = pension_plan_purch['pk_partition'] - pd.DateOffset(months=1)


In [236]:
pension_plan_purch.head()

,pk_cid,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan,pk_part_prev_month
71,1050659,2018-01-01,KFC,1,02 - PARTICULARES,7,V,39,-1.00,1,2017-12-01
120,1050565,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,33,54141.03,1,2017-12-01
140,1050537,2018-01-01,KHE,1,03 - UNIVERSITARIO,50,H,23,214309.14,1,2017-12-01
186,1050869,2018-01-01,KHE,1,03 - UNIVERSITARIO,28,V,31,45758.82,1,2017-12-01
222,1050920,2018-01-01,KHE,1,03 - UNIVERSITARIO,46,H,25,-1.00,1,2017-12-01


In [242]:
pension_plan_purch = pension_plan_purch[['pk_cid', 'pk_part_prev_month']]

In [244]:
# Mergeamos por pk_cid y pk_partition del mes anterior a compra con el dataframe de las no compras

df_customers_purchased = pd.merge(
    left = pension_plan_purch, 
    right = pension_plan_no_purch, 
    left_on = ('pk_cid', 'pk_part_prev_month'), 
    right_on=('pk_cid', 'pk_partition'),
    how='inner'
    )

In [245]:
pension_plan_no_purch .info()

<class 'pandas.core.frame.DataFrame'>
Index: 1039859 entries, 0 to 1047518
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype         
---  ------            --------------    -----         
 0   pk_cid            1039859 non-null  int64         
 1   pk_partition      1039859 non-null  datetime64[ns]
 2   entry_channel     1039859 non-null  object        
 3   active_customer   1039859 non-null  int64         
 4   segment           1039859 non-null  object        
 5   region_code       1039859 non-null  int64         
 6   gender            1039859 non-null  object        
 7   age               1039859 non-null  int64         
 8   salary            1039859 non-null  float64       
 9   reg_pension_plan  1039859 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(3)
memory usage: 87.3+ MB


In [247]:
df_customers_purchased.head(10)

,pk_cid,pk_part_prev_month,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,456665,2018-01-01,2018-01-01,KAT,1,03 - UNIVERSITARIO,8,V,32,145200.78,0
1,492727,2018-01-01,2018-01-01,KHE,1,02 - PARTICULARES,46,V,36,37690.02,0
2,485371,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,8,H,41,107403.51,0
3,806854,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,28,V,43,116528.10,0
4,797955,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,15,V,46,-1.00,0
5,758840,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,28,H,47,101766.36,0
6,865312,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,28,V,51,255480.66,0
7,859000,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,28,H,60,68652.06,0
8,873806,2018-01-01,2018-01-01,KFC,1,02 - PARTICULARES,8,V,55,53373.15,0
9,850255,2018-01-01,2018-01-01,KAT,1,02 - PARTICULARES,41,V,81,217192.89,0


In [248]:
df_customers_purchased.shape

(5451, 11)

Para tener un modelo balanceado hacen falta clientes que no hayan comprado ni vayan a comprar. Para eso, hacemos un outer merge con el df de los clientes que han comprado, según el pk_cid. Quedandonos así con el pk_cid de clientes que no coinciden en ninguna de las tablas (es decir, con los pk_cid que no tengan compra de pension plan)

In [260]:
df_customers_no_purchased=pd.merge(
    left = pension_plan_purch, 
    right = pension_plan_no_purch, 
    left_on = ('pk_cid', 'pk_part_prev_month'), 
    right_on=('pk_cid', 'pk_partition'),
    how='outer'
    )

In [261]:
df_customers_no_purchased

,pk_cid,pk_part_prev_month,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,15891,NaT,2018-07-01,KAT,1.0,Not defined,28.0,H,59.0,-1.00,0.0
1,15891,NaT,2018-08-01,KAT,0.0,02 - PARTICULARES,28.0,H,59.0,-1.00,0.0
2,16063,NaT,2018-11-01,KAT,1.0,Not defined,28.0,H,62.0,-1.00,0.0
3,16063,NaT,2018-12-01,KAT,1.0,02 - PARTICULARES,28.0,H,62.0,-1.00,0.0
4,16063,NaT,2019-01-01,KAT,1.0,02 - PARTICULARES,28.0,H,62.0,-1.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1042063,1079478,NaT,2019-02-01,KHE,0.0,03 - UNIVERSITARIO,28.0,H,24.0,93954.99,0.0
1042064,1079478,NaT,2019-03-01,KHE,0.0,03 - UNIVERSITARIO,28.0,H,24.0,93954.99,0.0
1042065,1079478,NaT,2019-04-01,KHE,0.0,03 - UNIVERSITARIO,28.0,H,24.0,93954.99,0.0
1042066,1079478,NaT,2019-05-01,KHE,0.0,03 - UNIVERSITARIO,28.0,H,24.0,93954.99,0.0


De aquí tendremos que seleccionar, al azar, la misma cantidad de filas de aquellos clientes que compraron, para un modelo balanceado. En total, hacen falta 5451 que no hayan comprado.

Seleccionar 5451 clientes del no-purchase  
Concatenar df df_customers_purchased + df_customers_no_purchased  
Eliminar pk_partition_pre_month  

In [271]:
df_customers_no_purchased_sample = df_customers_no_purchased.sample(n=5451, random_state=42)

In [272]:
df_customers_no_purchased_sample.shape

(5451, 11)

## Concatenamos los dataframes 

Unimos los clientes de los dataframes para tener nuestro dataset para entrenar el modelo.

In [279]:
df = pd.concat([df_customers_purchased, df_customers_no_purchased_sample], axis=0)

In [280]:
df

,pk_cid,pk_part_prev_month,pk_partition,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,456665,2018-01-01,2018-01-01,KAT,1.0,03 - UNIVERSITARIO,8.0,V,32.0,145200.78,0.0
1,492727,2018-01-01,2018-01-01,KHE,1.0,02 - PARTICULARES,46.0,V,36.0,37690.02,0.0
2,485371,2018-01-01,2018-01-01,KAT,1.0,02 - PARTICULARES,8.0,H,41.0,107403.51,0.0
3,806854,2018-01-01,2018-01-01,KAT,1.0,02 - PARTICULARES,28.0,V,43.0,116528.10,0.0
4,797955,2018-01-01,2018-01-01,KAT,1.0,02 - PARTICULARES,15.0,V,46.0,-1.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...
439659,1034924,NaT,2018-12-01,KHE,0.0,03 - UNIVERSITARIO,10.0,V,29.0,-1.00,0.0
827789,1064577,NaT,2018-05-01,KHE,0.0,03 - UNIVERSITARIO,10.0,H,23.0,46819.71,0.0
540981,1042314,NaT,2018-12-01,KHE,0.0,03 - UNIVERSITARIO,41.0,H,23.0,89575.50,0.0
659385,1050894,NaT,2018-02-01,KHE,1.0,03 - UNIVERSITARIO,28.0,H,25.0,92941.65,0.0


### Eliminamos columnas no útiles: pk_part_prev_month, pk_partition y pk_cid

In [282]:
df.drop(columns=['pk_part_prev_month','pk_partition', 'pk_cid'], inplace=True)

In [283]:
df

,entry_channel,active_customer,segment,region_code,gender,age,salary,reg_pension_plan
0,KAT,1.0,03 - UNIVERSITARIO,8.0,V,32.0,145200.78,0.0
1,KHE,1.0,02 - PARTICULARES,46.0,V,36.0,37690.02,0.0
2,KAT,1.0,02 - PARTICULARES,8.0,H,41.0,107403.51,0.0
3,KAT,1.0,02 - PARTICULARES,28.0,V,43.0,116528.10,0.0
4,KAT,1.0,02 - PARTICULARES,15.0,V,46.0,-1.00,0.0
...,...,...,...,...,...,...,...,...
439659,KHE,0.0,03 - UNIVERSITARIO,10.0,V,29.0,-1.00,0.0
827789,KHE,0.0,03 - UNIVERSITARIO,10.0,H,23.0,46819.71,0.0
540981,KHE,0.0,03 - UNIVERSITARIO,41.0,H,23.0,89575.50,0.0
659385,KHE,1.0,03 - UNIVERSITARIO,28.0,H,25.0,92941.65,0.0


1. Sustituir region code por nombre de ciudades

In [ ]:
# Diccionario que mapea los códigos de región a nombres
codigo_a_nombre = {
    1: 'Alava',
    2: 'Albacete',
    3: 'Alicante',
    4: 'Almeria',
    5: 'Avila',
    6: 'Badajoz',
    7: 'Islas Baleares',
    8: 'Barcelona',
    9: 'Burgos',
    10: 'Caceres',
    11: 'Cadiz',
    12: 'Castellon',
    13: 'Ciudad Real',
    14: 'Cordoba',
    15: 'La Coruña',
    16: 'Cuenca',
    17: 'Gerona',
    18: 'Granada',
    19: 'Guadalajara',
    20: 'Guipuzcoa',
    21: 'Huelva',
    22: 'Huesca',
    23: 'Jaen',
    24: 'Leon',
    25: 'Lerida',
    26: 'La Rioja',
    27: 'Lugo',
    28: 'Madrid',
    29: 'Malaga',
    30: 'Murcia',
    31: 'Navarra',
    32: 'Orense',
    33: 'Asturias',
    34: 'Palencia',
    35: 'Las Palmas',
    36: 'Pontevedra',
    37: 'Salamanca',
    38: 'Santa Cruz de Tenerife',
    39: 'Cantabria',
    40: 'Segovia',
    41: 'Sevilla',
    42: 'Soria',
    43: 'Tarragona',
    44: 'Teruel',
    45: 'Toledo',
    46: 'Valencia',
    47: 'Valladolid',
    48: 'Vizcaya',
    49: 'Zamora',
    50: 'Zaragoza',
    51: 'Ceuta',
    52: 'Melilla'
}

# Reemplaza los valores en la columna 'region_code' usando el diccionario
df['region_code'] = df['region_code'].replace(codigo_a_nombre)

# Muestra el DataFrame para verificar los cambios
print(df)

2. hacer OHE de entry_channel, segment, gender, nombre ciudad 

for col in col_to_OHE:
    _dummy_dataset = pd.get_dummies(df_mmp[col], prefix=col)
    df_mmp = pd.concat([df_mmp,_dummy_dataset],axis=1)
    df_mmp.drop([col],axis=1, inplace=True)